反卷积网络，通过测量输出和已知输入重构未知输入的过程。在神经网络中，反卷积过程并不具备学习的能力，仅仅是**用于可视化**一个已经训练好的卷积网络模型，没有学习训练的过程。

下图为VGG 16反卷积神经网络结构，展示了一个卷积网络与反卷积网络结合的过程。VGG 16是一个深度神经网络模型。它的反卷积就是将中间的数据，按照前面卷积、池化等变化的过程，完全相反地做一遍，从而得到类似原始输入的数据。
![](imgs/19_vgg16.png)

## 反卷积神经网络应用场景
由于反卷积网络的特性，导致它有许多特别的应用，一般可以用于**信道均衡、图像恢复、语音识别、地震学、无损探伤**等未知输入估计和过程辨别方面的问题。

在神经网络的研究中，反卷积更多是冲淡可视化的作用。对于一个复杂的深度卷积网络，通过每层若干个卷积核的变换，我们无法知道每个卷积核关注的是什么，变换后的特征是什么样子。通过**反卷积的还原**，可以对这些问题有个清晰的可视化，以各层得到的特征图作为输入，进行反卷积，得到反卷积结果，用以验证显示各层提取到的特征图。

## 反卷积原理
反卷积，可以理解为卷积操作的逆操作。这里千万不要当成反卷积操作可以复原卷积操作的输入值，反卷积并没有这个功能，它仅仅是将卷积变换过程中的步骤反向变换一个而已，通过将卷积核转置，与卷积后的结果再做一遍卷积，所以它还有个名字叫**转置卷积**。
虽然它不能还原出原来卷积的样子，但是在作用上具有类似的效果，可以将带有小部分缺失的信息最大化地恢复，也可以用来恢复被卷积生成后的原始输入。

反卷积具体操作比较复杂，具体步骤如下：
1. 首先是将卷积核反转（并不是转置，而是上下左右方向进行递序操作）。
2. 再将卷积结果作为输入，做补0的扩充操作，即往每一个元素后面补0.这一步是根据步长来的，对每一个元素沿着步长的方向补（步长-1）个0.例如，步长为1就不用补0了。
3. 在扩充后的输入基础上再对整体补0.以原始输入的shape作为输出，按照前面卷积padding规则，计算padding的补0位置及个数，得到的补0位置要上下和左右各自颠倒一下。
4. 将补0后的卷积结果作为真正的输入，翻转后的卷积核为filter，进行步长为1的卷积操作。

> 计算padding按规则补0 时，统一按照padding='SAME'、步长为1x1的方式来计算。


以一个[1,4,4,1]矩阵为例，进行filter为2x2，步长为2x2的卷积操作，反卷积操作如下图：![](imgs/19_deconvolution.png)

在反卷积过程中，首先将2x2矩阵通过步长补0的方式变成4x4，再通过padding反方向补0，然后与反转后的filter使用步长为1x1的卷积操作，最终得出结果。但是这个结果已经与原来的全1矩阵不等了，说明转置卷积只能回复部分特征，无法百分百地恢复原始数据。

## 演示反卷积操作
在TensorFlow中反卷积是通过函数tf.nn.conv2d_transpose来实现的，定义如下：
___
```python
conv2d_transpose(value,
                filter,
                output_shape,
                strides,
                padding='SAME',
                data_format='NHWC',
                name=None)
```
___
- value:代表通过卷积操作之后的张量，一般用NHWC类型。
- filter:代表卷积核。
- output_shape:代表输出的张量形状也是个四维张量。
- padding：代表原数据生成value时用的补0的方式，是用来检查输入形状和输出形状是否合规的。
- strides:步长。
- return : 反卷积后的结果，按照output_shape指定的形状。

> NHWC类型是神经网络中在处理图像方面常用的类型，4个字母分别代表4个意思，即N-个数、H-高、W-宽度、C-通道数。也就是常见的四维张量。

> output_shape并**不是一个随便写的形状**，它必须是能够生成value参数的元数据的形状，如果输出形状不对，函数会报错。
反卷积操作其实是使用了gen_nn_ops.conv2d_backprop_input函数来最终实现的，相当于TensorFlow中利用了卷积操作在反向传播的处理函数中做反卷积操作，即卷积操作的反向传播就是反卷积操作。

### 实例描述
通过模拟数据进行卷积与反卷积的操作，来比较卷积与反卷积中padding在SAME、VALID下的变化。

In [2]:
import numpy as np
import tensorflow as tf

#模拟数据
img = tf.Variable(tf.constant(1.0,shape=[1,4,4,1]))

filter = tf.Variable(tf.constant([1.0,0,-1,-2],shape=[2,2,1,1]))

#分别进行VALID 和 SAME操作
conv = tf.nn.conv2d(img,filter=filter,strides=[1,2,2,1],padding='VALID')
cons = tf.nn.conv2d(img,filter,strides=[1,2,2,1],padding='SAME')
print(conv.shape)
print(cons.shape)

#进行反卷积
contv = tf.nn.conv2d_transpose(conv,filter,[1,4,4,1],strides=[1,2,2,1],padding='VALID')
conts = tf.nn.conv2d_transpose(cons,filter,[1,4,4,1],strides=[1,2,2,1],padding='SAME')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print("conv:\n",sess.run([conv,filter]))
    print("cons:\n",sess.run([cons]))
    print("contv:\n",sess.run([contv]))
    print("conts:\n",sess.run([conts]))    

(1, 2, 2, 1)
(1, 2, 2, 1)
conv:
 [array([[[[-2.],
         [-2.]],

        [[-2.],
         [-2.]]]], dtype=float32), array([[[[ 1.]],

        [[ 0.]]],


       [[[-1.]],

        [[-2.]]]], dtype=float32)]
cons:
 [array([[[[-2.],
         [-2.]],

        [[-2.],
         [-2.]]]], dtype=float32)]
contv:
 [array([[[[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]],

        [[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]]]], dtype=float32)]
conts:
 [array([[[[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]],

        [[-2.],
         [ 0.],
         [-2.],
         [ 0.]],

        [[ 2.],
         [ 4.],
         [ 2.],
         [ 4.]]]], dtype=float32)]


先定义一个[1,4,4,1]的矩阵，矩阵里的值全为1，进行filter为2x2、步长为2x2的卷积操作，分别使用padding为SAME和VALID的两种情况生成卷积数据，然后将结果放到tf.nn.conv2d_transpose里，再次使用padding为SAME和VALID的两种情况生成数据。可以看出，输出的结果与上图是一样的，并且也验证了当padding为SAME并不需要补0时，卷积和反卷积对于padding是SAME和VALID都是相同的。

# 反池化原理
反池化是属于池化的逆操作，是无法通过池化的结果还原出全部的原始数据。因为池化的过程就是**只保留主要信息，舍去部分信息**。如想从池化后的这些主要信息恢复出全部信息，则存在着信息缺失，这时只能通过补位来实现最大程度的信息完整。

池化有两种最大池化和平均池化，其反池化也需要与其对应。

- 平均池化的操作比较简单。首先还原成原来的大小，然后将池化结果中的每个值都填入其对应于原始数据区域中的相应位置即可，如下图：
![](imgs/19_avg_pooling.png)

- 最大池化的反池化会复杂一些。要求在池化过程中记录**最大激活值的坐标位置**，然后在反池化时，只把池化过程中最大激活值所在位置坐标的值激活，其他的位置为0.当然，这个过程只是一种近似。因为在池化过程中，除了最大值所在的位置，其他的值也是不为0的。![](imgs/19_max_anti_pooling.png)

## 演示反池化操作
TensorFlow还没有反池化操作的函数。对于最大池化，也不支持输出最大激活值的位置，但是同样有个池化的反向传播函数tf.nn.max_pool_with_argmax。该函数可以输出位置，需要开发者利用这个函数做一些改动，自己封装一个最大池化操作，然后再根据mask写出反池化函数。

### 实例描述
定义一个数组作为模拟图片，将其进行最大池化，接着再进行反池化，比较原始数据与反池化后的数据。

定义最大池化函数：
___
```python
def max_pool_with_argmax(net,stride):
    _,mask = tf.nn.max_pool_with_argmax(net,ksize=[1,stride,stride,1],stride=[1,stride,stride,1],padding='SAME')
    mask=tf.stop_gradient(mask)
    net = tf.nn.max_pool(net,ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
    return net,mask
```
___
先调用tf.nn.max_pool_with_argmax函数获得每个最大值的位置mask，再将反向传播的nask梯度计算停止，接着再用tf.nn.max_pool函数计算最大池化操作，然后将mask和池化结果一起返回。
定义一个数组，并使用最大池化函数对其进行池化操作，比较一下与自带的tf.nn.pool函数是否一样。

In [14]:
def max_pool_with_argmax(net,stride):
    _,mask = tf.nn.max_pool_with_argmax(net,ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
    mask=tf.stop_gradient(mask)
    net = tf.nn.max_pool(net,ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
    return net,mask

img = tf.constant([  
       [[0.0,4.0],[0.0,4.0],[0.0,4.0],[0.0,4.0]],  
       [[1.0,5.0],[1.0,5.0],[1.0,5.0],[1.0,5.0]],  
       [[2.0,6.0],[2.0,6.0],[2.0,6.0],[2.0,6.0]],  
       [[3.0,7.0],[3.0,7.0], [3.0,7.0],[3.0,7.0]]
     ])  
img = tf.reshape(img,[1,4,4,2])
pooling2 = tf.nn.max_pool(img,[1,2,2,1],[1,2,2,1],padding='SAME')
encode,mask = max_pool_with_argmax(img,2)
img2 = unpool(encode,mask,2)

# tf.reset_default_graph()

with tf.Session() as sess:
    print("image:")
    image = sess.run(img)
    print(image)
    
    result = sess.run(pooling2)
    print("pooling2:\n",result)

    result,mask2 = sess.run([encode,mask])
    print("encode:\n",result,mask2)
    
    print("encode:\n",result,mask2)
    result = sess.run(img2)
    print("result:\n",result)
    

image:
[[[[0. 4.]
   [0. 4.]
   [0. 4.]
   [0. 4.]]

  [[1. 5.]
   [1. 5.]
   [1. 5.]
   [1. 5.]]

  [[2. 6.]
   [2. 6.]
   [2. 6.]
   [2. 6.]]

  [[3. 7.]
   [3. 7.]
   [3. 7.]
   [3. 7.]]]]
pooling2:
 [[[[1. 5.]
   [1. 5.]]

  [[3. 7.]
   [3. 7.]]]]
encode:
 [[[[1. 5.]
   [1. 5.]]

  [[3. 7.]
   [3. 7.]]]] [[[[ 8  9]
   [12 13]]

  [[24 25]
   [28 29]]]]
encode:
 [[[[1. 5.]
   [1. 5.]]

  [[3. 7.]
   [3. 7.]]]] [[[[ 8  9]
   [12 13]]

  [[24 25]
   [28 29]]]]
result:
 [[[[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[1. 5.]
   [0. 0.]
   [1. 5.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[3. 7.]
   [0. 0.]
   [3. 7.]
   [0. 0.]]]]


定义的最大池化与原来的版本输出是一样的。mask的值是将整个数组flat（扁平化）后的索引，但却保持与池化结果一致的shape。

In [13]:
# 反最大池化
def unpool(net,mask,stride):
    
    ksize = [1, stride, stride, 1]
    input_shape = net.get_shape().as_list()
    #  calculation new shape
    output_shape = (input_shape[0], input_shape[1] * ksize[1], input_shape[2] * ksize[2], input_shape[3])
    # calculation indices for batch, height, width and feature maps
    one_like_mask = tf.ones_like(mask)
    batch_range = tf.reshape(tf.range(output_shape[0], dtype=tf.int64), shape=[input_shape[0], 1, 1, 1])
    b = one_like_mask * batch_range
    y = mask // (output_shape[2] * output_shape[3])
    x = mask % (output_shape[2] * output_shape[3]) // output_shape[3]
    feature_range = tf.range(output_shape[3], dtype=tf.int64)
    f = one_like_mask * feature_range
    # transpose indices & reshape update values to one dimension
    updates_size = tf.size(net)
    indices = tf.transpose(tf.reshape(tf.stack([b, y, x, f]), [4, updates_size]))
    values = tf.reshape(net, [updates_size])
    ret = tf.scatter_nd(indices, values, output_shape)
    return ret

In [15]:
img2 = unpool(encode,mask,2)
with tf.Session() as sess:
    print("encode:\n",result,mask2)
    result = sess.run(img2)
    print("result:\n",result)

encode:
 [[[[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[1. 5.]
   [0. 0.]
   [1. 5.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[3. 7.]
   [0. 0.]
   [3. 7.]
   [0. 0.]]]] [[[[ 8  9]
   [12 13]]

  [[24 25]
   [28 29]]]]
result:
 [[[[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[1. 5.]
   [0. 0.]
   [1. 5.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[3. 7.]
   [0. 0.]
   [3. 7.]
   [0. 0.]]]]


最大值已经填入对应的位置，其他地方的值为0。